In [443]:
import pandas as pd
import pycountry
import plotly.express as px
import numpy as np
import plotly.io as pio

# Use plotly as the backend for pandas
pd.options.plotting.backend = "plotly"
pio.renderers.default = "plotly_mimetype+notebook_connected"


objects_in_orbit = pd.read_csv(
    '../input/ucs-satellites.txt', sep='\t', encoding="ISO-8859-1")
space_missions = pd.read_csv(
    "../input/space-missions.csv",  encoding="ISO-8859-1")

# ISO 3166-1 alpha-3 country codes for all known launch sites
country_codes = {"Kazakhstan": "RUS", "USA": "USA", "Russia": "RUS", "France": "FRA", "Japan": "JPN", "Kenya": "KEN", "Australia": "AUS", "China": "CHN", "India": "IND", "Israel": "ISR", "Gran Canaria": "ESP", "Brazil": "BRA",
                 "Barents Sea": "RUS", "North Korea": "PRK", "Pacific Ocean": "USA", "Iran": "IRN", "South Korea": "KOR", "Pacific Missile Range Facility": "USA", "New Zealand": "NZL", "New Mexico": "USA", "Yellow Sea": "CHN", "Shahrud Missile Test Site": "IRN", "Czech Republic": "CZE", "Vietnam": "VNM"}


# Exploring the UCS Objects In Orbit dataset


In [444]:
# Print the column names of the first 28 columns
objects_in_orbit.columns[:28]


Index(['Name of Satellite, Alternate Names',
       'Current Official Name of Satellite', 'Country/Org of UN Registry',
       'Country of Operator/Owner', 'Operator/Owner', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', ' Dry Mass (kg.) ', 'Power (watts)',
       'Date of Launch', 'Expected Lifetime (yrs.)', 'Contractor',
       'Country of Contractor', 'Launch Site', 'Launch Vehicle',
       'COSPAR Number', 'NORAD Number', 'Comments'],
      dtype='object')

In [445]:
# How many values are missing in each column out of the first 28?
objects_in_orbit.iloc[:, :28].isna().sum()


Name of Satellite, Alternate Names       2
Current Official Name of Satellite       2
Country/Org of UN Registry             499
Country of Operator/Owner                2
Operator/Owner                           2
Users                                    2
Purpose                                  2
Detailed Purpose                      4327
Class of Orbit                           2
Type of Orbit                          624
Longitude of GEO (degrees)               3
Perigee (km)                             3
Apogee (km)                              2
Eccentricity                             2
Inclination (degrees)                    2
Period (minutes)                        18
Launch Mass (kg.)                      242
 Dry Mass (kg.)                       5023
Power (watts)                         4887
Date of Launch                           2
Expected Lifetime (yrs.)              1868
Contractor                               4
Country of Contractor                    4
Launch Site

In [446]:
# Get the 11 desired columns (`Name of Satellite, Alternate Names', Country of Operator/Owner, Operator/Owner, Users,	Purpose, Perigee (km), Date of Launch, Expected Lifetime (yrs.), Contractor, Country of Contractor, Launch Site)
objects_in_orbit = objects_in_orbit[['Name of Satellite, Alternate Names', "Period (minutes)", 'Country of Operator/Owner', 'Operator/Owner', 'Users',
                                     'Purpose', 'Perigee (km)', 'Class of Orbit', 'Type of Orbit', 'Date of Launch', 'Expected Lifetime (yrs.)', 'Contractor', 'Country of Contractor', 'Launch Site']]


In [447]:
objects_in_orbit.shape


(5467, 14)

In [448]:
# Remove commas from the Perigee (km) column
objects_in_orbit['Perigee (km)'] = objects_in_orbit['Perigee (km)'].str.replace(
    ',', '').astype(float)


# set the perigee column to numeric
objects_in_orbit['Perigee (km)'] = pd.to_numeric(
    objects_in_orbit['Perigee (km)'], errors='coerce')

# set the longitude of geo column to numeric
objects_in_orbit['Period (minutes)'] = pd.to_numeric(
    objects_in_orbit['Period (minutes)'], errors='coerce')


In [449]:
objects_in_orbit['Users'].unique()


array(['Commercial', 'Civil', 'Government', 'Military',
       'Military/Commercial', 'Government/Military',
       'Military/Government', 'Government/Civil', 'Earth Observation',
       'Military/Civil', 'Commercial/Civil', 'Civil/Commercial',
       'Government/Commercial', 'Commercial/Government',
       'Government/Commercial/Military', 'Civil/Government',
       'Civil/Military', 'Commercial ', 'Commercial/Military',
       'Communications', 'Government ', 'Military ', nan], dtype=object)

In [450]:
# Clean up the users columns by replacing entries of the format A/B with A and removing training spaces
objects_in_orbit['Users'] = objects_in_orbit['Users'].str.replace(
    r'\/.*', '').str.strip()
# Replace nan users with commercial
objects_in_orbit['Users'] = objects_in_orbit['Users'].fillna('Commercial')


/var/folders/wn/sv8fngps4f5bfbddkt_d3y8h0000gn/T/ipykernel_12949/728387890.py:2: FutureWarning:

The default value of regex will change from True to False in a future version.



In [451]:
# Get the distinct users
objects_in_orbit['Users'].unique()


array(['Commercial', 'Civil', 'Government', 'Military',
       'Earth Observation', 'Communications'], dtype=object)

In [452]:
# Get the count of missing values per column
objects_in_orbit.isna().sum()


Name of Satellite, Alternate Names       2
Period (minutes)                        18
Country of Operator/Owner                2
Operator/Owner                           2
Users                                    0
Purpose                                  2
Perigee (km)                             3
Class of Orbit                           2
Type of Orbit                          624
Date of Launch                           2
Expected Lifetime (yrs.)              1868
Contractor                               4
Country of Contractor                    4
Launch Site                              3
dtype: int64

In [453]:
# Replace null values in the Period column with the average
objects_in_orbit['Period (minutes)'] = objects_in_orbit['Period (minutes)'].fillna(
    objects_in_orbit['Period (minutes)'].mean())

# Replace null values in the perigee column with the average
objects_in_orbit['Perigee (km)'] = objects_in_orbit['Perigee (km)'].fillna(
    objects_in_orbit['Perigee (km)'].mean())


In [454]:
# Unique values in the orbit class column
objects_in_orbit['Class of Orbit'].unique()


array(['LEO', 'GEO', 'Elliptical', 'MEO', nan], dtype=object)

In [455]:
# get the min, max and distribution of values in the Perigee (km) column
objects_in_orbit['Perigee (km)'].describe()


count     5467.000000
mean      4712.516837
std      10977.260267
min        150.000000
25%        496.000000
50%        548.000000
75%        706.000000
max      62200.000000
Name: Perigee (km), dtype: float64

In [456]:
objects_in_orbit.sample()


,"Name of Satellite, Alternate Names",Period (minutes),Country of Operator/Owner,Operator/Owner,Users,Purpose,Perigee (km),Class of Orbit,Type of Orbit,Date of Launch,Expected Lifetime (yrs.),Contractor,Country of Contractor,Launch Site
3358,Starlink-1643,95.6,USA,SpaceX,Commercial,Communications,548.0,LEO,Non-Polar Inclined,8/18/2020,NaN,SpaceX,USA,Cape Canaveral


In [457]:


# Replace purposes of the format A\/B with A
objects_in_orbit['Purpose'] = objects_in_orbit['Purpose'].str.replace(
    r'\/.*', '').str.strip()

# Replace nan in the purpose column with Other
objects_in_orbit['Purpose'] = objects_in_orbit['Purpose'].fillna('Other')

# Get the unique purposes
objects_in_orbit['Purpose'].unique()


/var/folders/wn/sv8fngps4f5bfbddkt_d3y8h0000gn/T/ipykernel_12949/3928839402.py:2: FutureWarning:

The default value of regex will change from True to False in a future version.



array(['Earth Observation', 'Technology Development', 'Communications',
       'Earth Science', 'Space Science', 'Technology Demonstration',
       'Unknown', 'Navigation', 'Signals Intelligence', 'Earth',
       'Educational', 'Platform', 'Space Observation', 'Surveillance',
       'Earth Observarion', 'Mission Extension Technology',
       'Amateur Radio', 'Satellite Positioning', 'Other'], dtype=object)

In [458]:

purposes_df = objects_in_orbit['Purpose'].value_counts().to_frame()
purposes_df = purposes_df.rename(columns={'Purpose': 'Count'})
# Rename the index to 'Purpose'
purposes_df.index.name = 'Purpose'
purposes_df = purposes_df.reset_index()

# Plot a bar chart of the purpose and count columns with 10 ticks on the y axis
purposes_df.head(5).plot.bar(x='Purpose', y='Count',
                             ).update_yaxes(nticks=10)


In [459]:
# Get the number of unique values in the Purpose column
objects_in_orbit['Purpose'].nunique()


19

In [460]:
# Convert the Date of Launch column to a datetime object
objects_in_orbit['Date of Launch'] = pd.to_datetime(
    objects_in_orbit['Date of Launch'])

# Create a dataframe from the year of the Date of Launch column
launch_years_df = objects_in_orbit['Date of Launch'].dt.year.value_counts(
).to_frame()
launch_years_df = launch_years_df.rename(columns={'Date of Launch': 'Count'})
# Rename the index to 'Year'
launch_years_df.index.name = 'Year'
launch_years_df = launch_years_df.reset_index()

# Plot a histogram of the launch years with 50 bins

launch_years_df.plot.hist(
    x='Year', y='Count',  nbins=50,  labels={
        "Count": "Count"
    }).update_layout(yaxis_title="Count").update_yaxes(nticks=10)


In [461]:
# Sort the launch years dataframe by year
launch_years_df = launch_years_df.sort_values(by='Year')
# Export launch_years_df as a json file named satellites_over_time.json, sorted by the Year column
launch_years_df.to_json(
    '../output/formatted_data/satellites_over_time.json', orient='records')


Most of the objects in orbit were launched in the last 5 years


In [462]:
# Get the percentage of satellites that were launched since 2020
last_5_years = objects_in_orbit[objects_in_orbit['Date of Launch']
                                >= '2020-01-01']
f"{last_5_years.shape[0] / objects_in_orbit.shape[0] * 100}% of satellites were launched since 2020"


'63.910737150173766% of satellites were launched since 2020'

In [463]:
# When was the first satellite launched?
print(
    f"The first satellite was launched on {objects_in_orbit['Date of Launch'].min()}")

# Sort the dataframe by the Date of Launch column
objects_in_orbit = objects_in_orbit.sort_values(by='Date of Launch')
objects_in_orbit.head(1)


The first satellite was launched on 1974-11-15 00:00:00


,"Name of Satellite, Alternate Names",Period (minutes),Country of Operator/Owner,Operator/Owner,Users,Purpose,Perigee (km),Class of Orbit,Type of Orbit,Date of Launch,Expected Lifetime (yrs.),Contractor,Country of Contractor,Launch Site
77,Amsat-Oscar 7 (AO-7),114.9,USA,AMSAT-NA,Civil,Communications,1440.0,LEO,Polar,1974-11-15,NaN,Amsat-NA,USA,Vandenberg AFB


In [464]:
# Get the histogram of values in the launch site column
objects_in_orbit['Launch Site'].value_counts().head()


Cape Canaveral          2719
Baikonur Cosmodrome      442
Guiana Space Center      380
Vandenberg AFB           338
Vostochny Cosmodrome     270
Name: Launch Site, dtype: int64

In [465]:


def get_country_code(country_name):
    try:
        return pycountry.countries.get(name=country_name).alpha_3
    except:
        if country_name in country_codes:
            return country_codes[country_name]
        return "Multinational"


# Map the country names to their 3-letter codes
objects_in_orbit['Country Code of Operator/Owner'] = objects_in_orbit['Country of Operator/Owner'].apply(
    get_country_code)
objects_in_orbit['Country Code of Operator/Owner'].value_counts()

# Plot a histogram of the top 10 countries (excluding rows where the code is Multinational)
country_counts = objects_in_orbit['Country Code of Operator/Owner'].value_counts()
country_counts[country_counts.index !=
               "Multinational"].head(10).plot(kind='bar')


In [466]:
# Get the histogram of values in the Country of Contractor column
objects_in_orbit['Country of Contractor'].value_counts().head()


USA       3745
China      540
Russia     165
UK         155
France      93
Name: Country of Contractor, dtype: int64

In [467]:
# How many satellites does spaceX own?
f"SpaceX own {objects_in_orbit[objects_in_orbit['Operator/Owner'] == 'SpaceX'].shape[0]} satellites"


'SpaceX own 2219 satellites'

In [468]:
# Get the histogram of values in the Operator/Owner column
objects_in_orbit['Operator/Owner'].value_counts().head()


SpaceX                                  2219
OneWeb Satellites                        427
Planet Labs, Inc.                        215
Swarm Technologies                       151
Chinese Ministry of National Defense     135
Name: Operator/Owner, dtype: int64

In [469]:
# Replace NaN Operator/Owner with Unknown
objects_in_orbit['Operator/Owner'] = objects_in_orbit['Operator/Owner'].fillna(
    'Unknown')


# Replace Ministry of Defense in Operator/Owner with Ministry of Defense (Country of Operator/Owner for that row)
objects_in_orbit['Operator/Owner'] = objects_in_orbit.apply(
    lambda row: row['Operator/Owner'] if row['Operator/Owner'] != 'Ministry of Defense' else f"Ministry of Defense ({row['Country of Operator/Owner']})", axis=1)
# Replace Country of Operator/Owner 'ESA' with 'Multinational'
objects_in_orbit['Country of Operator/Owner'] = objects_in_orbit['Country of Operator/Owner'].apply(
    lambda x: 'Multinational' if x == 'ESA' else x)

In [470]:
# Get the percentage of objects that belong to SpaceX
objects_in_orbit['Operator/Owner'].value_counts(normalize=True).head()


SpaceX                                  0.405890
OneWeb Satellites                       0.078105
Planet Labs, Inc.                       0.039327
Swarm Technologies                      0.027620
Chinese Ministry of National Defense    0.024694
Name: Operator/Owner, dtype: float64

In [471]:
# Get the first distinct "Longitude of GEO (degrees)" values
print(len(objects_in_orbit['Period (minutes)'].unique()))
objects_in_orbit['Period (minutes)'].head(5)


581


77       114.90
5074    1436.06
769     1436.13
1068      95.80
2569    1436.07
Name: Period (minutes), dtype: float64

In [472]:
# Plot a chart with perigee of all satellites on the y-axis
# Add an x column with a random value between 1 and 100
objects_in_orbit['x'] = np.random.randint(1, 1000, objects_in_orbit.shape[0])
# Sort the dataframe by the Perigee (km) column
objects_in_orbit = objects_in_orbit.sort_values(by='Perigee (km)')
# Print the type of the Perigee (km) column
print(type(objects_in_orbit['Perigee (km)'][0]))
# Plot a scatter plot of the x and Perigee (km) columns. Color the points by the Purpose column. Use plotly.
fig = px.scatter(objects_in_orbit, x='x', y='Perigee (km)', color='Users')
# Use a horizontal legend
fig.update_layout(legend=dict(orientation="h"))
# Hide the X axis tick labels
fig.update_xaxes(showticklabels=False)
# Hide the X axis title
fig.update_xaxes(title_text="")
fig.show()


<class 'numpy.float64'>


In [473]:
fig = px.scatter(objects_in_orbit, x='x', y='Perigee (km)', color='Purpose')
# Use a horizontal legend
fig.update_layout(legend=dict(orientation="h"))
# Hide the X axis tick labels
fig.update_xaxes(showticklabels=False)
# Hide the X axis title
fig.update_xaxes(title_text="")
fig.show()


In [474]:
# Plot a histogram of the perigee of all satellites using a logarithmic scale with plotly
fig = px.histogram(objects_in_orbit, x='Perigee (km)', nbins=100)
fig.show()


In [475]:
fig = px.scatter(objects_in_orbit, x='x',
                 y='Perigee (km)', color='Class of Orbit')
# Use a horizontal legend
fig.update_layout(legend=dict(orientation="h"))
# Hide the X axis tick labels
fig.update_xaxes(showticklabels=False)
# Hide the X axis title
fig.update_xaxes(title_text="")
fig.show()


In [476]:
# Plot a histogram of the top 5 operators
objects_in_orbit['Operator/Owner'].value_counts().head(5).plot(kind='bar')


In [477]:
# Plot a histogram of the top 5 launch countries
objects_in_orbit['Country of Operator/Owner'].value_counts().head(5).plot(kind='bar')


In [478]:
# Save objects_in_orbit['Operator/Owner'].value_counts() as a json file named satellite_owners.json
objects_in_orbit['Operator/Owner'].value_counts().head(5).to_json(
    '../output/formatted_data/satellite_owners.json', orient='index')

# Save objects_in_orbit['Country of Operator/Owner'].value_counts() as a json file named satellite_countries.json
objects_in_orbit['Country of Operator/Owner'].value_counts().head(5).to_json(
    '../output/formatted_data/satellite_countries.json', orient='index')


In [479]:
objects_in_orbit['Operator/Owner'].value_counts().head(10)


SpaceX                                       2219
OneWeb Satellites                             427
Planet Labs, Inc.                             215
Swarm Technologies                            151
Chinese Ministry of National Defense          135
Spire Global Inc.                             124
Ministry of Defense (Russia)                   96
Iridium Communications, Inc.                   75
Indian Space Research Organization (ISRO)      45
Chang Guang Satellite Technology Co. Ltd.      44
Name: Operator/Owner, dtype: int64

In [480]:
import json

# Get the top countries by number of satellites
countries = objects_in_orbit['Country of Operator/Owner'].value_counts()

N_COUNTRIES = 11
MIN_N_SATELLITES = 15

# Get the top satellite owners for each of the top countries
country_to_operator = [{"name":"Other", "children":[{"name":"Other", "value": 0}]}]
for country in countries.head(N_COUNTRIES).index:
    # Get the satellite owners for each country and add them to a children list in the format  {"name": satellite_owner, "value": number_of_satellites}
    counts = objects_in_orbit[objects_in_orbit['Country of Operator/Owner'] == country]['Operator/Owner'].value_counts()
    children = []

    for child in counts.index:
        if counts[child] >= MIN_N_SATELLITES:
            children.append({
                "name": child,
                "value": int(counts[child])
            })
        else:
            country_to_operator[0]['children'][0]['value'] += int(counts[child])
    
    if(len(children) > 0):
        country_to_operator.append({
            "name": country,
            "children": children
        })

country_to_operator[0]['children'][0]['value'] += int(objects_in_orbit[~objects_in_orbit['Country of Operator/Owner'].isin(countries.head(N_COUNTRIES).index)]['Operator/Owner'].value_counts().sum())
# Move the first value to the end
country_to_operator.append(country_to_operator.pop(0))


# Save country_to_operator as a json file named satellite_owners_by_country.json
with open('../output/formatted_data/satellite_owners_by_country.json', 'w') as f:
    json.dump({"children": country_to_operator}, f)

# Exploring the Space Missions dataset


In [481]:
# We only want the Company, Location, Date, Mission and Price columns
space_missions = space_missions[['Company',
                                 'Location', 'Date', 'Mission', 'Price']]


In [482]:
space_missions.head()


,Company,Location,Date,Mission,Price
0,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-10-04,Sputnik-1,NaN
1,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03,Sputnik-2,NaN
2,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06,Vanguard TV3,NaN
3,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01,Explorer 1,NaN
4,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05,Vanguard TV3BU,NaN


In [483]:
print(space_missions.shape)
space_missions.columns


(4630, 5)


Index(['Company', 'Location', 'Date', 'Mission', 'Price'], dtype='object')

In [484]:
space_missions_clean = space_missions.dropna()
space_missions_clean.sample()


,Company,Location,Date,Mission,Price
2485,NASA,"LC-39A, Kennedy Space Center, Florida, USA",1990-02-28,STS-36,450


In [485]:
print(f"There are {space_missions_clean.shape} clean rows")

# How many null values are there?
space_missions.isna().sum()


There are (1265, 5) clean rows


Company        0
Location       0
Date           0
Mission        0
Price       3365
dtype: int64

In [486]:
# Construct a datetime from the date and time columns
space_missions['Date'] = pd.to_datetime(space_missions['Date'])


In [487]:
def launches_over_time(df):
    df_freq = df['Date'].value_counts().rename_axis(
        'Date').reset_index(name='Frequency')
    df_freq.sort_values(by='Date',  ascending=True, inplace=True)
    df_freq["Cumulative Sum"] = df_freq["Frequency"].cumsum()
    return df_freq


## Historic launches


In [488]:

# Plot the number of space missions over time, with datetime on the X axis
space_missions_freq = launches_over_time(space_missions)
space_missions_freq.plot(x='Date', y='Cumulative Sum',
                         kind='line')


In [489]:
# Drop the Cumulative Sum column
space_missions_freq.drop(columns=['Cumulative Sum'], inplace=True)

# Rename the Frequency column to count
space_missions_freq.rename(columns={'Frequency': 'count'}, inplace=True)

# Group the dataframe by the year and sum the values
space_missions_freq = space_missions_freq.groupby(
    space_missions_freq['Date'].dt.year).sum()

# Fix the index
space_missions_freq.reset_index(inplace=True)

# Rename the Date column to 'year'
space_missions_freq.rename(columns={'Date': 'year'}, inplace=True)

space_missions_freq.head()


/var/folders/wn/sv8fngps4f5bfbddkt_d3y8h0000gn/T/ipykernel_12949/2215241199.py:8: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,year,count
0,1957,3
1,1958,28
2,1959,20
3,1960,39
4,1961,52


In [490]:
space_missions_freq.to_json(
    '../output/formatted_data/missions_over_time.json', orient='records')


## SpaceX launches


In [491]:
# Plot the number of SpaceX space missions over time, with datetime on the X axis
space_missions_spacex = space_missions[space_missions['Company'] == 'SpaceX']
space_missions_spacex_freq = launches_over_time(space_missions_spacex)
space_missions_spacex_freq.plot(
    x='Date', y='Cumulative Sum', kind='line', title="SpaceX Missions Over Time")


In [492]:
# get the values of pycountry.countries
country_names = [country.name for country in pycountry.countries]
country_names


['Aruba',
 'Afghanistan',
 'Angola',
 'Anguilla',
 'Åland Islands',
 'Albania',
 'Andorra',
 'United Arab Emirates',
 'Argentina',
 'Armenia',
 'American Samoa',
 'Antarctica',
 'French Southern Territories',
 'Antigua and Barbuda',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Burundi',
 'Belgium',
 'Benin',
 'Bonaire, Sint Eustatius and Saba',
 'Burkina Faso',
 'Bangladesh',
 'Bulgaria',
 'Bahrain',
 'Bahamas',
 'Bosnia and Herzegovina',
 'Saint Barthélemy',
 'Belarus',
 'Belize',
 'Bermuda',
 'Bolivia, Plurinational State of',
 'Brazil',
 'Barbados',
 'Brunei Darussalam',
 'Bhutan',
 'Bouvet Island',
 'Botswana',
 'Central African Republic',
 'Canada',
 'Cocos (Keeling) Islands',
 'Switzerland',
 'Chile',
 'China',
 "Côte d'Ivoire",
 'Cameroon',
 'Congo, The Democratic Republic of the',
 'Congo',
 'Cook Islands',
 'Colombia',
 'Comoros',
 'Cabo Verde',
 'Costa Rica',
 'Cuba',
 'Curaçao',
 'Christmas Island',
 'Cayman Islands',
 'Cyprus',
 'Czechia',
 'Germany',
 'Djibouti',
 'Dominica'

In [493]:
# Get the launch country as everything after the last comma in the location column
space_missions['Launch Country Code'] = space_missions['Location'].str.split(
    ',').str[-1].map(lambda country: get_country_code(country.strip()))


# Use pycountry to get the country name from the 3-letter code
space_missions['Launch Country'] = space_missions['Launch Country Code'].map(
    lambda code: pycountry.countries.get(alpha_3=code).name)


space_missions.head()


,Company,Location,Date,Mission,Price,Launch Country Code,Launch Country
0,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-10-04,Sputnik-1,NaN,KAZ,Kazakhstan
1,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03,Sputnik-2,NaN,KAZ,Kazakhstan
2,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06,Vanguard TV3,NaN,USA,United States
3,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01,Explorer 1,NaN,USA,United States
4,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05,Vanguard TV3BU,NaN,USA,United States


In [494]:
# ' Plot a bar chart of the Launch Country column
space_missions['Launch Country'].value_counts().plot(
    kind='bar', title="Space Missions by Launch Country")


In [495]:
# Plot a bar chart of the launch company column for the top 10 companies
space_missions['Company'].value_counts().head(10).plot(
    kind='bar', title="Space Missions by Launch Company")


In [496]:
# Get all the space missions before 1970
space_race_missions = space_missions[space_missions['Date'] < '1965-01-01']


In [497]:
# Plot a choropleth map of the number of space missions by launch country (before 1965)
fig = px.choropleth(space_race_missions, locations="Launch Country Code",
                    color="Launch Country Code", hover_name="Launch Country", color_continuous_scale=px.colors.sequential.Plasma)
fig.update_layout(
    title_text='Space Missions by Launch Country (Pre-1965)',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)


In [498]:
# Plot a choropleth map of the number of space missions by launch country
fig = px.choropleth(space_missions, locations="Launch Country",
                    color="Launch Country", hover_name="Launch Country", color_continuous_scale=px.colors.sequential.Plasma)
fig.update_layout(
    title_text='Space Missions by Launch Country (to date)',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)


In [499]:
# Remove the 'x' column from objects_in_orbit
objects_in_orbit = objects_in_orbit.drop(columns=['x'])

objects_in_orbit_json_columns = {
    "Class of Orbit": "orbitClass",
    "Contractor": "contractor",
    "Country Code of Operator/Owner": "ownerCountryCode",
    "Country of Operator/Owner": "ownerCountry",
    "Country of Contractor": "contractorCountry",
    "Date of Launch": "launchDate",
    "Expected Lifetime (yrs.)": "expectedLifetime",
    "Launch Site": "launchSite",
    "Name of Satellite, Alternate Names": "name",
    "Operator/Owner": "owner",
    "Perigee (km)": "perigee",
    "Period (minutes)": "period",
    "Purpose": "purpose",
    "Type of Orbit": "orbitType",
    "Users": "users"
}

# Rename the columns of objects_in_orbit
objects_in_orbit = objects_in_orbit.rename(
    columns=objects_in_orbit_json_columns)


In [500]:
space_missions_json_columns = {
    "Company": "company",
    "Date": "date",
    "Location": "location",
    "Mission": "mission",
    "Price": "price",
    "Launch Country": "country",
    "Launch Country Code": "countryCode",
    "Frequency": "missions"
}


In [501]:
# Get the number of space missions by launch country each year

# When exported, the JSON format should be [{year: 2020, 'usa': 20, 'russia': 0}]

def get_missions_by_country_by_year(df):
    # Replace Launch Country Code values where the value is KAZ with RUS
    df['Launch Country'] = df['Launch Country'].replace(
        'Kazakhstan', 'Russia')

    # Replace Russian Federation with Russia
    df['Launch Country'] = df['Launch Country'].replace(
        'Russian Federation', 'Russia')

    # Get the top 5 countries by number of space missions
    top_countries = df['Launch Country'].value_counts().head(5).index.tolist()

    # Replace all country codes not in the top 5 with 'Other'
    df['Launch Country'] = df['Launch Country'].map(
        lambda country: country if country in top_countries else 'Rest of world')

    # Get the number of space missions by launch country each year
    df = df.groupby([df['Date'].dt.year, 'Launch Country']
                    ).size().reset_index(name='count')
    # Pivot the dataframe so that the countries are the columns
    df = df.pivot(index='Date', columns='Launch Country', values='count')
    # Fill the NaN values with 0
    df.fillna(0, inplace=True)
    return df


missions_by_country_by_year = get_missions_by_country_by_year(
    space_missions).rename(columns=space_missions_json_columns)

# Reset the index
missions_by_country_by_year.reset_index(inplace=True)

# Rename the Date column to year
missions_by_country_by_year.rename(columns={'Date': 'year'}, inplace=True)

missions_by_country_by_year


Launch Country,year,China,France,Japan,Rest of world,Russia,United States
0,1957,0.0,0.0,0.0,0.0,2.0,1.0
1,1958,0.0,0.0,0.0,0.0,5.0,23.0
2,1959,0.0,0.0,0.0,0.0,4.0,16.0
3,1960,0.0,0.0,0.0,0.0,9.0,30.0
4,1961,0.0,0.0,0.0,0.0,9.0,43.0
...,...,...,...,...,...,...,...
61,2018,39.0,11.0,6.0,10.0,17.0,34.0
62,2019,34.0,9.0,2.0,15.0,22.0,27.0
63,2020,39.0,7.0,4.0,12.0,15.0,42.0
64,2021,55.0,7.0,3.0,11.0,24.0,57.0


In [502]:
# Download the missions_by_country_by_year dataframe as a JSON file. It should be in the format [{year: 2020, 'usa': 20, 'russia': 0}]
missions_by_country_by_year.to_json(
    '../output/formatted_data/missions_by_country_by_year.json', orient='records')


In [503]:
# Get the number of space missions launched by Russia by 1965
russia_missions_by_year = get_missions_by_country_by_year(
    space_missions[space_missions['Date'] < '1965-01-01'])
russia_missions_by_year['Russia'].sum()

/var/folders/wn/sv8fngps4f5bfbddkt_d3y8h0000gn/T/ipykernel_12949/2397675144.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/wn/sv8fngps4f5bfbddkt_d3y8h0000gn/T/ipykernel_12949/2397675144.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/wn/sv8fngps4f5bfbddkt_d3y8h0000gn/T/ipykernel_12949/2397675144.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

111

In [504]:


# We need to get data in the format country, number of missions to date for a given year
# We can do this by grouping by country and year, and then getting the cumulative sum of the number of missions
def get_missions_by_country(df, year):
    df = df[df['Date'] < year]

    df = df.groupby(['Launch Country', 'Launch Country Code',
                    'Date']).size().reset_index(name='Frequency')
    df = df.groupby(['Launch Country', 'Launch Country Code']).agg(
        {'Frequency': 'sum'}).reset_index()
    df.sort_values(by='Frequency', ascending=False, inplace=True)
    return df


space_missions_agg = get_missions_by_country(
    space_missions, '2023-01-01').rename(columns=space_missions_json_columns)
space_race_missions_agg = get_missions_by_country(
    space_missions, "1965-01-01").rename(columns=space_missions_json_columns)

# Rename the columns of space_missions
space_missions = space_missions.rename(columns=space_missions_json_columns)


In [505]:
# Write the aggregate data
space_missions_agg.to_json(
    '../output/formatted_data/space_missions_2022.json', orient='records')
space_race_missions_agg.to_json(
    "../output/formatted_data/space_missions_1965.json", orient="records")


# Write satellites to satellites.csv
objects_in_orbit.to_json(
    '../output/formatted_data/satellites.json', orient="records")
